In [49]:
options(warn=-1)
library(readr)
suppressWarnings(library(dplyr))
library(xlsx)

In [11]:
#KODA
likvidnost <- function(datum, par) {
    ime <- paste(datum, '/', par, '.csv', sep='')
    data <- read.csv(ime, header=T)
    data$date2 <- as.POSIXct(data$date,origin="1970-01-01",tz="GMT") 
    tmp <- split(data, cut(strptime(data$date2, format="%F %R"),"hour"))
    df <- data.frame(Date=as.Date(as.character()), buy=numeric(), sell=numeric(), stringsAsFactors=FALSE)
    for (ura in tmp) {
        tmpbuy <- ura[ura[, "type"] == 0,]
        tmpsell <- ura[ura[, "type"] == 1,]
        vec <- c(ura$date2[[1]], sum(tmpbuy$amount), sum(tmpsell$amount))
        df <- rbind(df, vec)
    }
    colnames(df)<- c("date","buy","sell")
    df$difference <- df$buy - df$sell
    df$date <- as.POSIXct(df$date,origin="1970-01-01",tz="GMT")
    df$daily_sum <- cumsum(df$difference)
    df <- df[1:24,]
    return(df)
}

In [61]:
#KODA
rez <- data.frame(Date=as.Date(as.character()), buy=numeric(), sell=numeric(), difference=numeric(),
                                                                              daily_sum=numeric(),stringsAsFactors=FALSE)
for (i in 13:36) {
    if (i < 31) {
        datum <- paste(as.character(i), ".09.2017", sep="")
    } else {
        i <- (i - 30)
        datum <- paste("0", as.character(i), ".10.2017", sep="")
    }
    tmp <- likvidnost(datum, 'btcusd')
    rez <- rbind(rez, tmp)
}



daily <- data.frame(Date=as.Date(as.character()), daily_sum=numeric())

for (i in 1:24) {
    vec <- c(rez$date[i*24], rez$daily_sum[i*24])
    daily <- rbind(daily, vec)
}

colnames(daily)<- c("date","difference")
daily$date <- as.POSIXct(daily$date,origin="1970-01-01",tz="GMT")



#dim(rez)
#head(rez)

#dim(daily)
#head(daily)

In [62]:
write.xlsx(rez, "indikator_urni.xlsx", sheetName = "List1")

In [63]:
prices <- read.xlsx("btcusd_price.xlsx", sheetName = "List1", header=FALSE)
prices <- prices[,1:3]
prices <- prices[nrow(prices):1,]
colnames(prices)<- c("date","open", "close")
#prices

In [64]:
daily$price_diff <- prices$close - prices$open
colnames(daily) <- c("date", "flow_difference", "price_difference")
daily$date <- substring(as.character(daily$date),1,10)
daily

date,flow_difference,price_difference
2017-09-13,-3422.69803,-249.39
2017-09-14,-3142.75894,-720.42
2017-09-15,1380.82588,471.22
2017-09-16,153.80432,-12.71
2017-09-17,50.40402,-23.40
2017-09-18,2180.96892,474.11
2017-09-19,-1179.95979,-148.82
2017-09-20,-1081.33217,-10.41
2017-09-21,-1580.77906,-270.43
2017-09-22,656.36138,2.68


In [53]:
cor(daily$flow_difference, daily$price_difference)

[1] 0.7860662

In [68]:
write.xlsx(daily, "indikator_dnevni.xlsx", sheetName = "List1", row.names=FALSE)

In [67]:
?write.xlsx